# Running large problems on D-Wave QPUs using QBSolv

To solve a problem with a D-Wave QPU, one defines the problem in QUBO (or Ising) form. Owing to the limited qubit connectivity in D-Wave devices, the source graph given by the QUBO (or Ising) problem must be mapped to the target graph of the underlying hardware following a process called minor embedding. For the D-Wave 2000Q device with a Chimera graph, there is no guarantee to find an embedding for a problem with more than 64 variables; for the Advantage device with a Pegasus graph, the upper limit for an arbitrary problem is around 145. However, some real-world problems may require more variables than a QPU can support. To solve large problems with the D-Wave devices, you can use `QBSolv()`, which is offered by the Ocean SDK. 

`QBSolv()` is a hybrid solver that can decompose large QUBO problems into sub-problems. The sub-problems are solved using either both the QPU and a classical Tabu solver, or the classical solver alone. The solution to the problem is then constructed by the results of the sub-problems. More details can be found in the D-Wave documentation [here](https://docs.ocean.dwavesys.com/projects/qbsolv/en/latest/).

This notebook demonstrates how to run large problems with `QBsolv()` on Amazon Braket.

## General imports

In [1]:
import random
import itertools
from itertools import combinations
import networkx as nx
from collections import defaultdict
from dwave_qbsolv import QBSolv
from dwave.system.composites import FixedEmbeddingComposite
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler
import minorminer
import time

__NOTE__: Enter your desired device and S3 location (bucket and prefix). Remember that bucket names for Amazon Braket always begin with "amazon-braket-". 

In [2]:
# enter the S3 bucket you created during onboarding in the code below
my_bucket = "amazon-braket-Your-Bucket-Name" # the name of the bucket
my_prefix = "Your-Folder-Name" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

## Define the problem

`QBSolv()` accepts input models in QUBO, Ising or BQM forms, which correspond to `QBSolv().sample_qubo()`, `QBSolv().sample_ising()` and `QBSolv().sample()` methods respectively. This notebook takes a random QUBO problem as the example.

The following code generates a random graph and constructs a QUBO matrix as the problem to solve.  

In [3]:
# number of nodes
nodes = 240
# number of edges
edges = round(nodes * (nodes-1)/2 * 0.4)
seed = 2

# generate a random graph
Graph = nx.gnm_random_graph(nodes, edges, seed)

# Initialize the QUBO matrix
QUBO = defaultdict(int)

# Update the QUBO matrix for every edge in the graph
for u, v in Graph.edges:
    QUBO[(u,u)]+= -1
    QUBO[(v,v)]+= -3
    QUBO[(u,v)]+= 2

## Run tasks with QBSolv

You can specify the size of the sub-problems with the `solver_limit` parameter, and choose a solver to run the sub-problems. Note if the size of the problem you want to solve is smaller than this value, the task will run only with the Tabu solver. 

In [4]:
# define size of the sub-problems
solver_limit = 40

# choose the D-Wave Advantage system
system = BraketDWaveSampler(s3_folder, 'arn:aws:braket:::device/qpu/d-wave/Advantage_system1')

To run sub-problems on a D-Wave QPU, you must find an embedding to map the sub-problems to the graph of the device. The following example finds an embedding of a complete graph (thus is applicable to any problem of the same size), and provides the embedding to the solve using the `FixedEmbeddingComposite()` method. 

In [5]:
# find embedding of subproblem-sized complete graph to the QPU
G = nx.complete_graph(solver_limit)
embedding = minorminer.find_embedding(G.edges, system.edgelist)

# use the FixedEmbeddingComposite() method with a fixed embedding
solver = FixedEmbeddingComposite(system, embedding)

# define the number of repeats for the hybrid solver to search for the optimal solution.
num_repeats = 2

response = QBSolv().sample_qubo(QUBO, solver=solver, num_repeats=num_repeats, solver_limit=solver_limit)
print(response)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 239   energy num_oc.
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...   1 -25884.0       1
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...   1 -25884.0       2
['BINARY', 2 rows, 3 samples, 240 variables]


More information on the input parameters and a detailed description of the algorithm can be found [here](https://docs.ocean.dwavesys.com/projects/qbsolv/en/latest/intro.html).